# Pilot CFM implementation

### Load example programs


In [1]:
from example.python.crash import crashme, crashme3
from example.python.triangle import triangle, triangle3
from example.python.count import count

import inspect

program = crashme3
print(inspect.getsource(program))

def crashme3(s):
    if len(s) != 15:
        raise Exception("Wrong input.")
    if s[0] == "1":
        crashme(s[3:7])
    elif s[1] == "1":
        crashme(s[7:11])
    elif s[2] == "1":
        crashme(s[11:15])
    else:
        raise Exception("Wrong input.")



### Fuzzing configuration


In [2]:
n = 30000
seed_input_list = ["000abcdefghijkl", "100bad!asdfqwer", "010asdfbad!qwer", "001asdfqwerbad!"]
# seed_input_list = ["good", "bad"]
# seed_input_list = ["00012345good", "111123456bad"]
try:
    program(seed_input_list[1])
except Exception as e:
    print(e)

Deep bug!


### Run fuzzer and record input, coverage


In [3]:
from GreyboxFuzzer import GreyboxFuzzerRecorder
from MutationFuzzer import Mutator, PowerSchedule
from Coverage import FunctionContCovRunner

greybox_recorder = GreyboxFuzzerRecorder(
    seed_input_list, Mutator(), PowerSchedule()
)
runner = FunctionContCovRunner(program)
greybox_recorder.runs(runner, trials=n)
record = greybox_recorder.get_record()

print(f"Unique path: {len(record)}\n")
for path, inputs in list(record.items()):
    print(f"{path=}")
    print(f"{inputs=}")
    print()

Unique path: 17

path=(('', ('crashme3', 10)), ('', ('crashme3', 12)), ('', ('crashme3', 14)), ('', ('crashme3', 16)), ('', ('crashme3', 19)), ('', ('__exit__', 83)))
inputs={'080acdaSfghijkl', '00]1asdfqwebad!', '`00asdfbad!qwer', '00sddQi4l&biYd!', '00c0abcdefgijkl', '000abcdafghsihk', '0\x1eB1aslfs{pa"$!', '000abcdefghijkl', '0cM1\x00dfyrb?l`S!', 'q$|qp #i3fqse4R'}

path=(('', ('crashme3', 10)), ('', ('crashme3', 12)), ('', ('crashme3', 13)), ('crashme3:13', ('crashme', 2)), ('crashme3:13', ('crashme', 3)), ('crashme3:13', ('crashme', 4)), ('crashme3:13', ('crashme', 5)), ('crashme3:13', ('crashme', 6)), ('', ('__exit__', 83)))
inputs={'100bad!asdvqw%r', '110bad!asfdqver', '100bad!aqd&qwer', '100bad!as$fqwEr', '100bad!asdfqwer', '100bad!`sdfqwe2', '100bad!asdqwejr', '100bad!{asdfqwr', '100bad!sdfqkwer', '100bad!asdfxwer'}

path=(('', ('crashme3', 10)), ('', ('crashme3', 12)), ('', ('crashme3', 14)), ('', ('crashme3', 15)), ('crashme3:15', ('crashme', 2)), ('crashme3:15', ('crashme',

### Create a graph from coverage

In [4]:
from graph import Graph

contcov_graph = Graph()
coverage_graph = Graph()
for path, inputs in record.items():
    contcov_graph.accept(path)
    coverage_list = [contcov[1] for contcov in path]
    coverage_graph.accept(coverage_list)
# contcov_graph.print_graph()
# coverage_graph.print_graph()

In [5]:
from ControlFlowModel import ControlFlowModel

CFM = ControlFlowModel(record, contcov_graph, coverage_graph, runner)
CFM.identify_branch()
CFM.identify_call_context()

branches: [<('crashme3', 10)>, <('crashme3', 12)>, <('crashme3', 14)>, <('crashme3', 16)>, <('crashme', 2)>, <('crashme', 3)>, <('crashme', 4)>, <('crashme', 5)>]
call contexts: {'crashme3:17', 'crashme3:15', 'crashme3:13'}


### [Temp] assign temporary contextmap -- need to be changed to learn this

In [6]:
# context_map = {}
# context_map["crashme3:13"] = range(3, 7)
# context_map["crashme3:15"] = range(7, 11)
# context_map["crashme3:17"] = range(11, 15)

# CFM.set_context_map(context_map)

CFM.model_context()

[D] Analyze path=(('', ('crashme3', 10)), ('', ('crashme3', 12)), ('', ('crashme3', 14)), ('', ('crashme3', 16)), ('', ('crashme3', 19)), ('', ('__exit__', 83)))
[D] Sample input: 00c0abcdefgijkl
[D]    New input: 90c0abcdefgijkl
[D]    New input: 0%c0abcdefgijkl
[D]    New input: 0030abcdefgijkl
[D]    New input: 00c7abcdefgijkl
[D]    New input: 00c09bcdefgijkl
[D]    New input: 00c0aWcdefgijkl
[D]    New input: 00c0ab-defgijkl
[D]    New input: 00c0abcpefgijkl
[D]    New input: 00c0abcd8fgijkl
[D]    New input: 00c0abcdeYgijkl
[D]    New input: 00c0abcdefiijkl
[D]    New input: 00c0abcdefg)jkl
[D]    New input: 00c0abcdefgi6kl
[D]    New input: 00c0abcdefgijLl
[D]    New input: 00c0abcdefgijkA
[D] Analyze path=(('', ('crashme3', 10)), ('', ('crashme3', 12)), ('', ('crashme3', 13)), ('crashme3:13', ('crashme', 2)), ('crashme3:13', ('crashme', 3)), ('crashme3:13', ('crashme', 4)), ('crashme3:13', ('crashme', 5)), ('crashme3:13', ('crashme', 6)), ('', ('__exit__', 83)))
[D] Sample inpu

/Users/bohrok/.pyenv/versions/3.9.7/envs/fuzzingbook/lib/python3.9/site-packages/pulp/pulp.py:1313: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


### Model formula

In [7]:
CFM.model_condition()

Modeling <('crashme3', 10)> -> <('crashme3', 12)>... formula: <lambda x: x[14] != a> (conf: 0.9647058823529412)
Modeling <('crashme3', 10)> -> <('crashme3', 11)>... formula: <lambda x: x[3] != x[15]> (conf: 0.9411764705882353)
Modeling <('crashme3', 12)> -> <('crashme3', 14)>... formula: <lambda x: x[0] != 1> (conf: 1.0)
Modeling <('crashme3', 12)> -> <('crashme3', 13)>... formula: <lambda x: x[0] == 1> (conf: 1.0)
Modeling <('crashme3', 14)> -> <('crashme3', 16)>... formula: <lambda x: x[1] != 1> (conf: 1.0)
Modeling <('crashme3', 14)> -> <('crashme3', 15)>... formula: <lambda x: x[1] == 1> (conf: 1.0)
Modeling <('crashme3', 16)> -> <('crashme3', 19)>... formula: <lambda x: x[2] != 1> (conf: 1.0)
Modeling <('crashme3', 16)> -> <('crashme3', 17)>... formula: <lambda x: x[2] == 1> (conf: 1.0)
Modeling <('crashme', 2)> -> <('crashme', 3)>... formula: <lambda x: x[0] == b> (conf: 1.0)
Modeling <('crashme', 2)> -> <('__exit__', 83)>... No accepts or rejects; skip.
Modeling <('crashme', 3)>

In [8]:
CFM.get_context_map()

{'crashme3:15': range(7, 11),
 'crashme3:13': range(3, 7),
 'crashme3:17': range(11, 15)}

In [9]:
for edge, cond in CFM.get_edge_cond().items():
    print(f"edge={edge} {cond=}")

edge=E<('crashme3', 10) -> ('crashme3', 12)> cond=('lambda x: x[14] != a', 0.9647058823529412)
edge=E<('crashme3', 10) -> ('crashme3', 11)> cond=('lambda x: x[3] != x[15]', 0.9411764705882353)
edge=E<('crashme3', 12) -> ('crashme3', 14)> cond=('lambda x: x[0] != 1', 1.0)
edge=E<('crashme3', 12) -> ('crashme3', 13)> cond=('lambda x: x[0] == 1', 1.0)
edge=E<('crashme3', 14) -> ('crashme3', 16)> cond=('lambda x: x[1] != 1', 1.0)
edge=E<('crashme3', 14) -> ('crashme3', 15)> cond=('lambda x: x[1] == 1', 1.0)
edge=E<('crashme3', 16) -> ('crashme3', 19)> cond=('lambda x: x[2] != 1', 1.0)
edge=E<('crashme3', 16) -> ('crashme3', 17)> cond=('lambda x: x[2] == 1', 1.0)
edge=E<('crashme', 2) -> ('crashme', 3)> cond=('lambda x: x[0] == b', 1.0)
edge=E<('crashme', 2) -> ('__exit__', 83)> cond=(None, None)
edge=E<('crashme', 3) -> ('crashme', 4)> cond=('lambda x: x[1] == a', 1.0)
edge=E<('crashme', 3) -> ('__exit__', 83)> cond=(None, None)
edge=E<('crashme', 4) -> ('crashme', 5)> cond=('lambda x: x[2